# Module 3 Project 4: DSPy
- Implement a basic example using [DSPy](https://dspy-docs.vercel.app/) to get comfortable with how it works
- Note: this notebook should be run using a GPU and with `python3.9` - it also requires [TGI](https://github.com/huggingface/text-generation-inference) from HF to be running locally (alternatively, you can use your HuggingFace API key by uncommenting below)


In [ ]:
!pip install dspy-ai

## STEP 1: IMPORTS
- We need to import `dspy` and our dataset + metric evals for [GSM8K](https://huggingface.co/datasets/gsm8k) (Grade School Math 8k)
- We also need to import our optimizer and evaluator

In [ ]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate import Evaluate

## STEP 2: LOAD THE MODEL
- We load the model in this step
- We will be using [TheBloke's Mistral-7B-Instruct-v0.2-GPTQ](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GPTQ) model for this
- Configure dspy to use the model's parameters

In [ ]:
# Two options for using Huggingface, though the non-TGI version has some known bugs generating multiple line answers
llama = dspy.HFClientTGI(model="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", port=8080, url="http://localhost")
#llama = dspy.HFModel(model = 'TheBloke/Mistral-7B-Instruct-v0.2-GPTQ')

dspy.settings.configure(lm=llama)

## STEP 3: GSM8K DATASET
- Grade School Math (8K) is a dataset of math problems and answers 
- Answers include reasoning traces and calulations to help guide the model towards computation
- We only take the first 10 examples from each set for this demo

In [ ]:
# Load our dataset and split into train and validation
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

## STEP 4: SIMPLE DSPY MODULE
- Next, we create our simple [dspy.Module](https://dspy-docs.vercel.app/api/category/modules)
- It consists of one `ChainOfThought` layer with the signature: question -> answer
- The signature above denotes that there is a 'question' being asked, and that the model needs to respond with the 'answer' 
- Providing the 'question' from GSM8k leads to the model attempting to answer it

In [ ]:
# Create a DSPy module for CoT - we have 1 signature that takes in a question and returns an answer
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    # Forward pass prints the answer
    def forward(self, question):
        resp = self.prog(question=question)
        print(resp)
        return resp

## STEP 5: COMPILE
- Now we can compile our module created above into a dspy program
- Using out config of max 4 bootstrapped demos per run, and 4 maximum labeled demos
- We use `BootstrapFewShotWithRandomSearch` optimizer with the `gsm8k_metric` validation metric to measure accuracy over the dataset questions
- Lastly, we can use our optimizer to compile our module with our data

In [ ]:
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimizer is BootstrapFewShotWithRandomSearch as this is best for this sample size
teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)

# Compile our module with our data
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset, valset=gsm8k_devset)

## STEP 6: EVALUATE
- The final step here is to evaluate over our data
- We set 1 thread here, and run the evaluation
- Our optimizer will loop over a set # of seeds, re-iterating over the data and trying to tune for accuracy

In [ ]:
# Set up evaluation step
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=1, display_progress=True, display_table=0)

# Evaluate and display
evaluate(optimized_cot)